In [ ]:
from geopandas.tools import sjoin
import os
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import sys


from pathlib import Path
# Get the current working directory
current_dir = Path.cwd()

wor_dir = current_dir / Path('../../')




### Source for pre disaster indicators: https://data.humdata.org/dataset/philippines-pre-disaster-indicators

In [ ]:
file_name = "./data/pre_disaster_indicators/190108_pantawid-pamilya-beneficiary-households-and-household-members-by-sex-and-age-group.xlsx"

path = os.path.join(wor_dir, file_name)

pantawid_pamilya_beneficiary = pd.read_excel(path, sheet_name="Data", engine="openpyxl").filter(['PCODE_MUN_CTY','Total # of Active HHs','Total HH MEM_M','Total HH MEM_F'])
#pantawid_pamilya_beneficiary['Municipality_City Code']=pantawid_pamilya_beneficiary['PCODE_MUN_CTY']

pantawid_pamilya_beneficiary.rename(columns={"PCODE_MUN_CTY": "Mun_Code"},inplace=True)
pantawid_pamilya_beneficiary['pantawid_total_pop']=pantawid_pamilya_beneficiary["Total HH MEM_M"]+pantawid_pamilya_beneficiary["Total HH MEM_F"]
pantawid_pamilya_beneficiary =  pantawid_pamilya_beneficiary.filter(['Mun_Code','Total # of Active HHs','pantawid_total_pop'])
display(pantawid_pamilya_beneficiary.head())
 

 


### population per admin 

In [ ]:
file_name = "./data/pre_disaster_indicators/180814_philippines-population-admin-1-to-3_2015_vertical.xlsx"

path = os.path.join(wor_dir, file_name)

population_admin = pd.read_excel(path, sheet_name="Municipality", engine="openpyxl").filter(['Municipality_City Code','Female','Male']).groupby('Municipality_City Code').agg('sum')

population_admin['Total Pop']=population_admin.sum(axis="columns") 
population_admin['Mun_Code']=population_admin.index.tolist()

population_admin.reset_index(drop=True, inplace=True)

#population_admin.rename(columns={"Municipality_City Code": "Mun_Code"},inplace=True)
display(population_admin.head())

### vulnerable Groups

In [ ]:
file_name = "./data/pre_disaster_indicators/180814_vulnerable-groups_by-city_municipality.xlsx"

path = os.path.join(wor_dir, file_name)

vulnerable_groups = pd.read_excel(path, sheet_name="Data", engine="openpyxl")

vulnerable_groups['vulnerable_groups']=vulnerable_groups.sum(axis="columns")
vulnerable_groups=vulnerable_groups.filter(['Municipality_City Code','vulnerable_groups'])
vulnerable_groups.rename(columns={"Municipality_City Code": "Mun_Code"},inplace=True)
display(vulnerable_groups.head())

### housing material

In [ ]:
file_name = "./data/pre_disaster_indicators/180814_construction-materials-of-the-outer-walls-and-roof_by-city_municipality.xlsx"

path = os.path.join(wor_dir, file_name)

construction_materials = pd.read_excel(path, sheet_name="by category", engine="openpyxl")
construction_materials.rename(columns={"Municipality_City Code": "Mun_Code"},inplace=True)
construction_materials=construction_materials.filter(['Mun_Code', 'Housing Units','Strong Roof/Strong Wall', 'Strong Roof/Light Wall','Strong Roof/Salvage Wall', 'Light Roof/Strong Wall','Light Roof/Light Wall', 'Light Roof/Salvage Wall','Salvaged Roof/Strong Wall', 'Salvaged Roof/Light Wall', 'Salvaged Roof/Salvage Wall'])

df_cons_mat=construction_materials.iloc[:,2:].div(0.01*construction_materials["Housing Units"], axis=0)
df_cons_mat['Mun_Code']=construction_materials['Mun_Code']
df_cons_mat["Housing Units"]=construction_materials["Housing Units"]

display(df_cons_mat.head())


In [97]:
#df_cm=construction_materials
#df_cm.rename(columns ={'Housing Units':'HUNITS', 'Strong Roof/Strong Wall':'SRSW','Strong Roof/Light Wall':'SRLW', 'Strong Roof/Salvage Wall':'SRSW','Light Roof/Strong Wall':'LRSW', 'Light Roof/Light Wall':'LRLW','Light Roof/Salvage Wall':'LRSW', 'Salvaged Roof/Strong Wall':'SRSW','Salvaged Roof/Light Wall':'SRLW', 'Salvaged Roof/Salvage Wall':'SRSW'},inplace=True)


In [ ]:
len(construction_materials['Housing Units'])

### Data on topography related variables 

In [ ]:
geo_variable = pd.read_csv(./"data/topography/geo_variable.csv")
geo_variable=geo_variable.filter(['Mun_Code', 'mean_slope', 'mean_elevation_m','ruggedness_stdev', 'mean_ruggedness', 'slope_stdev', 'area_km2','poverty_perc', 'with_coast', 'coast_length', 'perimeter'])

display(geo_variable.head())

### Data for Land slide /storm surge vulnerable areas 

In [ ]:
data_matrix_new_variables = pd.read_csv("./data/landuse_stormsurge/data_matrix_new_variables.csv")
data_matrix_new_variables=data_matrix_new_variables.filter(['Mun_Code', 'landslide_per', 'stormsurge_per', 'Bu_p_inSSA','Bu_p_LS', 'Red_per_LSbldg', 'Or_per_LSblg', 'Yel_per_LSSAb','RED_per_SSAbldg', 'OR_per_SSAbldg', 'Yellow_per_LSbl'])
display(data_matrix_new_variables.head())

### Impact Data(Damage and Loss data)

In [152]:

#df_total["perc_loss"] = df_total.apply(lambda x: division(x["area_affected"], x["rice_area"]), axis=1).values
impact = pd.read_csv("./data/Impact/IMpact_data_philipines_SEP_2021.csv")
impact['typhoon']=impact["typhoon"].astype(str)  + impact["Year"].astype(str)
impact['typhoon']= impact['typhoon'].map(lambda x: x.lower())
impact.rename(columns={"pcode": "Mun_Code"},inplace=True)
typhoons_with_impact_data=list(np.unique(impact.typhoon))
#display(impact.head())
 
impact=impact[['Mun_Code','typhoon','Totally','Partially','total']]
df_impact=impact[['typhoon','Totally']].groupby('typhoon').sum().sort_values(by='Totally',ascending=False)#.head(n=10)

df_impact.reset_index(inplace=True)
df_impact['typhoon_name']=df_impact['typhoon'].apply(lambda x:(x[:-4].upper()+'-'+x[-4:]))

In [ ]:
display(impact.head())

### number of Households per Manucipality 

In [ ]:
file_name = "./data/pre_disaster_indicators/180814_number-of-household_by-city_municipality.xlsx"

path = os.path.join(wor_dir, file_name)

household_by_city_municipality = pd.read_excel(path, sheet_name="Data", engine="openpyxl").filter(['Municipality_City','Municipality_City Code','Number of Household'])
household_by_city_municipality.rename(columns={"Municipality_City Code": "Mun_Code"},inplace=True)
household_by_city_municipality=household_by_city_municipality.filter(['Mun_Code','Number of Household'])
display(household_by_city_municipality.head())


### historical Typhoons Methadata (name, start date, end date, land fall)

In [142]:
typhoon_metadata_filename = os.path.join(wor_dir, "./data/rainfall_data/input/metadata_typhoons.csv")
typhoon_metadata = pd.read_csv(typhoon_metadata_filename, delimiter=",")

typhoons = list(np.unique(impact.typhoon.values))# typhoon_metadata.typhoon.tolist() ## historical typhoon events 
municipalities=population_admin.Mun_Code.tolist() 

N_typh = len(typhoons)
N_mun = len(municipalities)

municipality_codes_full = np.repeat(municipalities, N_typh)
typhoons_full = typhoons * N_mun

In [ ]:
pantawid_pamilya_beneficiary

### create data frame for all pre disaster indicators 

In [ ]:
data_temp1 = {"Mun_Code": municipality_codes_full}#, "typhoon": typhoons_full}
df_temp_total = pd.DataFrame(data_temp1)

##data_matrix_new_variables
df_temp_total=df_temp_total.join(household_by_city_municipality.set_index('Mun_Code'), on='Mun_Code')

##data_matrix_new_variables
df_temp_total=df_temp_total.join(data_matrix_new_variables.set_index('Mun_Code'), on='Mun_Code')

#geo_variable
df_temp_total=df_temp_total.join(geo_variable.set_index('Mun_Code'), on='Mun_Code')
#construction_materials
df_temp_total=df_temp_total.join(df_cons_mat.set_index('Mun_Code'), on='Mun_Code')
#vulnerable_groups
df_temp_total=df_temp_total.join(vulnerable_groups.set_index('Mun_Code'), on='Mun_Code')
#population_admin
df_temp_total=df_temp_total.join(population_admin.set_index('Mun_Code'), on='Mun_Code')
#pantawid_pamilya_beneficiary
df_temp_total=df_temp_total.join(pantawid_pamilya_beneficiary.set_index('Mun_Code'), on='Mun_Code')

def housing(df):
    x=float(df[0])
    y=df[1]
    if np.isnan(y) and x>0:
        value=x
    elif np.isnan(y) and x<1:
        value=1
    else:
        value=y
    return value
df_temp_total["Housing Units"] = df_temp_total[['Number of Household','Housing Units']].apply(housing, axis="columns")
 
    
df_predisasters=df_temp_total#.set_index(['Mun_Code', 'typhoon'])


 

colnames=['landslide_per',
       'stormsurge_per', 'Bu_p_inSSA', 'Bu_p_LS', 'Red_per_LSbldg',
       'Or_per_LSblg', 'Yel_per_LSSAb', 'RED_per_SSAbldg', 'OR_per_SSAbldg',
       'Yellow_per_LSbl', 'mean_slope', 'mean_elevation_m', 'ruggedness_stdev',
       'mean_ruggedness', 'slope_stdev', 'area_km2', 'poverty_perc',
       'with_coast', 'coast_length', 'perimeter',
       'Strong Roof/Strong Wall', 'Strong Roof/Light Wall',
       'Strong Roof/Salvage Wall', 'Light Roof/Strong Wall',
       'Light Roof/Light Wall', 'Light Roof/Salvage Wall',
       'Salvaged Roof/Strong Wall', 'Salvaged Roof/Light Wall',
       'Salvaged Roof/Salvage Wall', 'vulnerable_groups', 'Female', 'Male',
       'Total Pop', 'Total # of Active HHs', 'Total HH MEM_M',
       'Total HH MEM_F']

for column in df_predisasters:
    if column in colnames:
        df_predisasters[column].fillna((df_predisasters[column].min(skipna=True)), inplace=True)
        
        
display(df_predisasters.head())

In [ ]:
df_predisasters.columns

In [ ]:
df_predisasters

### Add Typhoon wind Data

In [15]:
#Adding the wind data

data_temp1 = {"Mun_Code": municipality_codes_full, "typhoon": typhoons_full}
df_total = pd.DataFrame(data_temp1)

df_total["vmax"] = ""
df_total["dis_track_min"] = ""

df_wind_final= pd.DataFrame(columns=["typhoon", "Mun_Code", "v_max","dis_track_min"])

for typhoon in typhoons:

    # Path to the rainfall excel sheet per typhoon
    wind_path = os.path.join(
        wor_dir,
        "data/wind_data/output",
        typhoon +
        "_windgrid_output.csv",
    )
    df_temp = pd.read_csv(wind_path)
    
    df_wind_single = pd.DataFrame({"Mun_Code": df_temp["adm3_pcode"],"v_max": df_temp["v_max"],
                                   "dis_track_min": df_temp["dis_track_min"] })
    df_wind_single['typhoon']=typhoon
    df_wind_final = df_wind_final.append(df_wind_single) 


In [17]:
df_wind_final['typhoon_name']=df_wind_final['typhoon'].apply(lambda x:(x[:-4].upper()+'-'+x[-4:]))
 
df_wind=df_wind_final[['typhoon_name','v_max']].groupby('typhoon_name').max().sort_values(by='v_max',ascending=False)#.head(n=10)
df_wind.reset_index(inplace=True)


### Add rainfall data

In [18]:
# Path to the rainfall excel sheet per typhoon
rain_path = os.path.join(wor_dir,"./data/rainfall_data/rainfall_data.csv")
df_rain_final = pd.read_csv(rain_path)
df_rain_final=df_rain_final[df_rain_final.typhoon.isin(typhoons)]


### Merge rainfall and Wind data

In [ ]:
df_hazard = pd.merge(df_rain_final, df_wind_final,  how='left', left_on=['Mun_Code', 'typhoon'], right_on = ['Mun_Code', 'typhoon'])
#df_hazard=df_hazard[df_hazard['dis_track_min']<100]
display(df_hazard.head())  

In [ ]:
df_impact_hazard =pd.merge(df_hazard, impact,  how='left', left_on=['Mun_Code', 'typhoon'], right_on = ['Mun_Code', 'typhoon'])
display(df_impact_hazard.head()) 

In [ ]:
df_impact_hazard[['typhoon','rainfall_Total','v_max','Totally', 'rainfall_max_6h',
       'rainfall_max_24h']].groupby('typhoon').agg(rainfall_Total=('rainfall_Total', max), 
                                                   rainfall_max_6h=('rainfall_max_6h', max), 
                                                    rainfall_max_24h=('rainfall_max_24h', max), 
                                                   v_max=('v_max', max), 
                                                   Totally=('Totally', sum)).sort_values(by='rainfall_Total',ascending=False)


In [ ]:
df_impact_hazard['typhoon_name']=df_impact_hazard['typhoon'].apply(lambda x:(x[:-4].upper()+'-'+x[-4:]))
 
df_im_haz=df_impact_hazard[['typhoon_name','rainfall_Total','v_max','Totally']].groupby('typhoon_name').agg(rainfall_Total=('rainfall_Total', max), v_max=('v_max', max), Totally=('Totally', sum)).sort_values(by='Totally',ascending=False).head(n=20)
 
df_im_haz.rename(columns={"rainfall_Total": "Total Rainfall", "v_max": "Maximum Wind", "Totally": "Damaged Buildings"},inplace=True)

import matplotlib.pyplot as plt
import seaborn as sns
# Initialize the figure with a logarithmic x axis
plt.rcParams.update({'font.size': 12})
f, ax = plt.subplots(figsize=(7, 7))
ax.set(yscale="linear", xscale="linear")

sns.regplot (y='Damaged Buildings', x='Maximum Wind', data=df_im_haz,scatter_kws={"s": 100},order=1,ci=0)  
# Plot the orbital period with horizontal boxes
#sns.relplot(y='Damaged Buildings', x='Maximum Wind', data=df_im_haz,palette="vlag")

# Add in points to show each observation
#sns.stripplot(y='Models', x='MAPE', data=df_im_haz,size=2, color=".3", linewidth=0)

# Tweak the visual presentation
ax.xaxis.grid(True)
#ax.set(ylabel="")
ax.set_title('Comparsion of reported Damage with wind speed',fontsize=16)
ax.set_xlabel('Windspeed(m/s)',fontsize=14)
ax.set_ylabel('Number of Damaged Building',fontsize=12)
#ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
#sns.despine(trim=True, left=True)
#f.savefig('algorithms_comparsion_mape.jpg', format='jpg', dpi=600)
ax.yaxis.set_label_position("right")
f.savefig('wind_BUILDINGDAMAGE.jpg', format='jpg', dpi=600)
#fig.savefig('Hazard_damage_comparsion.jpg', format='jpg', dpi=400)

In [ ]:
f, ax = plt.subplots(figsize=(7, 7))
ax.set(yscale="linear", xscale="linear")

sns.regplot (y='Damaged Buildings', x='Total Rainfall', data=df_im_haz,scatter_kws={"s": 100},order=1,ci=0)  
# Plot the orbital period with horizontal boxes
#sns.relplot(y='Damaged Buildings', x='Maximum Wind', data=df_im_haz,palette="vlag")

# Add in points to show each observation
#sns.stripplot(y='Models', x='MAPE', data=df_im_haz,size=2, color=".3", linewidth=0)

# Tweak the visual presentation
ax.xaxis.grid(True)
#ax.set(ylabel="")
ax.set_title('Comparsion of reported Damage with Total Rainfall',fontsize=16)
ax.set_xlabel('Total Rainfall(mm)',fontsize=14)
ax.set_ylabel('Number of Damaged Building',fontsize=12)
ax.yaxis.set_label_position("right")
#ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
#sns.despine(trim=True, left=True)
f.savefig('RAIN_BUILDINGDAMAGE.jpg', format='jpg', dpi=600)


#fig.savefig('Hazard_damage_comparsion.jpg', format='jpg', dpi=400)


### Merge hazard(rainfall and Wind) data with Impac

In [ ]:
df_impact_hazard['typhoon_name']=df_impact_hazard['typhoon'].apply(lambda x:(x[:-4].upper()+'-'+x[-4:]))
 
df_im_haz=df_impact_hazard[['typhoon_name','rainfall_Total','v_max','Totally']].groupby('typhoon_name').agg(rainfall_Total=('rainfall_Total', max), v_max=('v_max', max), Totally=('Totally', sum)).sort_values(by='Totally',ascending=False).head(n=20)
 
def f(s):
    rmalized_df=(s-s.min())/(s.max()-s.min())
    return rmalized_df#s/s.max()

df_im_haz=df_im_haz[['rainfall_Total','v_max','Totally']].apply(f, axis=0)

df_im_haz.rename(columns={"rainfall_Total": "Total Rainfall", "v_max": "Maximum Wind", "Totally": "Damaged Buildings"},inplace=True)
df_im_haz
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(figsize=(16, 14))

 
df_im_haz.plot(ax=ax,kind='barh',color=['#6baed6','#bababa','#fb6a4a'])

ax.set_title('Comparsion of reported Damage with Hazard indicators',fontsize=18)
ax.set_xlabel('Normalized Windspeed, Rainfall and Total Nmber of Damaged Buildings',fontsize=18)
ax.set_ylabel('',fontsize=14)


fig.savefig('Hazard_damage_comparsion.jpg', format='jpg', dpi=400)




#
    

In [ ]:
df_total=pd.merge(df_impact_hazard, df_predisasters,  how='left', left_on=['Mun_Code', 'typhoon'], right_on = ['Mun_Code', 'typhoon'])
df_total=df_total[df_total['v_max'].notnull()]
df_total=df_total[df_total['rainfall_max_6h'].notnull()] 
#df_total=df_total[df_total['dis_track_min']<100]
display(df_total)



###  Normalize data uisng population size or number of housing uints per manucipality (Calculate percentage per manucipality)

In [ ]:

def division(x, y):
    try:
        value =100* (x / y)
        
    except:
        value = np.nan
    
    return 100 if value>100 else value


  

# Setting the new damage threshold

df_total["perc_dmg"] = df_total.apply(lambda x: division(x["Totally"], x["Housing Units"]), axis=1).values



df_total["vulnerable_groups"] = df_total.apply(lambda x: division(x["vulnerable_groups"], x["Total Pop"]), axis=1).values
#df_total["pantawid_pamilya_beneficiary"] = df_total.apply(lambda x: division(x["Total # of Active HHs"], x["Housing Units"]), axis=1).values
df_total["pantawid_pamilya_beneficiary"] = df_total.apply(lambda x: division(x["pantawid_total_pop"], x["Total Pop"]), axis=1).values
#df_total["Strong Roof/Strong Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Strong Wall"], x["Housing Units"]), axis=1).values
#df_total["Strong Roof/Light Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Light Wall"], x["Housing Units"]), axis=1).values
#df_total["Strong Roof/Salvage Wall"] = df_total.apply(lambda x: division(x["Strong Roof/Salvage Wall"], x["Housing Units"]), axis=1).values
#df_total["Light Roof/Strong Wall"] = df_total.apply(lambda x: division(x["Light Roof/Strong Wall"], x["Housing Units"]), axis=1).values
#df_total["Light Roof/Light Wall"] = df_total.apply(lambda x: division(x["Light Roof/Light Wall"], x["Housing Units"]), axis=1).values
##df_total["Light Roof/Salvage Wall"] = df_total.apply(lambda x: division(x["Light Roof/Salvage Wall"], x["Housing Units"]), axis=1).values
#df_total["Salvaged Roof/Strong Wall"] = df_total.apply(lambda x: division(x["Salvaged Roof/Strong Wall"], x["Housing Units"]), axis=1).values
#df_total["Salvaged Roof/Light Wall"] = df_total.apply(lambda x: division(x["Salvaged Roof/Light Wall"], x["Housing Units"]), axis=1).values
#df_total["Salvaged Roof/Salvage Wall"] = df_total.apply(lambda x: division(x["Salvaged Roof/Salvage Wall"], x["Housing Units"]), axis=1).values

display(df_total)

### filter final data frame for only relevant features 

In [41]:
df_total=df_total.filter(['Mun_Code', 'typhoon', 'rainfall_Total', 'rainfall_max_6h',
                 'rainfall_max_24h', 'v_max', 'dis_track_min','landslide_per', 'stormsurge_per',
                 'Bu_p_inSSA','Bu_p_LS', 'Red_per_LSbldg', 'Or_per_LSblg',
                 'Yel_per_LSSAb','RED_per_SSAbldg', 'OR_per_SSAbldg', 
                 'Yellow_per_LSbl', 'mean_slope','mean_elevation_m', 
                 'ruggedness_stdev', 'mean_ruggedness','slope_stdev',
                 'poverty_perc', 'with_coast', 'coast_length','Housing Units', 'Strong Roof/Strong Wall',
                 'Strong Roof/Light Wall', 'Strong Roof/Salvage Wall',
                 'Light Roof/Strong Wall', 'Light Roof/Light Wall',
                 'Light Roof/Salvage Wall', 'Salvaged Roof/Strong Wall',
                 'Salvaged Roof/Light Wall', 'Salvaged Roof/Salvage Wall',
                 'vulnerable_groups', 'pantawid_pamilya_beneficiary','perc_dmg'])


df_total.rename(columns ={"rainfall_Total":"HAZ_rainfall_Total",
                                     'rainfall_max_6h':'HAZ_rainfall_max_6h',
                                     'rainfall_max_24h':'HAZ_rainfall_max_24h',
                                     'v_max':'HAZ_v_max',
                                     'dis_track_min':'HAZ_dis_track_min',
                                     'binary_dmg':'DAM_binary_dmg',
                                     'perc_dmg':'DAM_perc_dmg',
                                    'landslide_per':'GEN_landslide_per',
                                    'stormsurge_per':'GEN_stormsurge_per',
                                    'Bu_p_inSSA':'GEN_Bu_p_inSSA',
                                    'Bu_p_LS':'GEN_Bu_p_LS',
                                     'Red_per_LSbldg':'GEN_Red_per_LSbldg',
                                    'Or_per_LSblg':'GEN_Or_per_LSblg',
                                     'Yel_per_LSSAb':'GEN_Yel_per_LSSAb',
                                    'RED_per_SSAbldg':'GEN_RED_per_SSAbldg',
                                     'OR_per_SSAbldg':'GEN_OR_per_SSAbldg',
                                    'Yellow_per_LSbl':'GEN_Yellow_per_LSbl',
                                     'mean_slope':'TOP_mean_slope',
                                    'mean_elevation_m':'TOP_mean_elevation_m',
                                     'ruggedness_stdev':'TOP_ruggedness_stdev',
                                    'mean_ruggedness':'TOP_mean_ruggedness',
                                     'slope_stdev':'TOP_slope_stdev',
                                     'poverty_perc':'VUL_poverty_perc',
                                    'with_coast':'GEN_with_coast',
                                     'coast_length':'GEN_coast_length',
                                     'Housing Units':'VUL_Housing_Units',
                                    'Strong Roof/Strong Wall':"VUL_StrongRoof_StrongWall",
                                    'Strong Roof/Light Wall':'VUL_StrongRoof_LightWall',
                                    'Strong Roof/Salvage Wall':'VUL_StrongRoof_SalvageWall',
                                    'Light Roof/Strong Wall':'VUL_LightRoof_StrongWall',
                                    'Light Roof/Light Wall':'VUL_LightRoof_LightWall',
                                    'Light Roof/Salvage Wall':'VUL_LightRoof_SalvageWall',
                                    'Salvaged Roof/Strong Wall':'VUL_SalvagedRoof_StrongWall',
                                    'Salvaged Roof/Light Wall':'VUL_SalvagedRoof_LightWall',
                                    'Salvaged Roof/Salvage Wall':'VUL_SalvagedRoof_SalvageWall',
                                    'vulnerable_groups':'VUL_vulnerable_groups',
                                    'pantawid_pamilya_beneficiary':'VUL_pantawid_pamilya_beneficiary'},inplace=True)




In [44]:
selected_columns=['Mun_Code', 'typhoon', 'HAZ_rainfall_Total', 'HAZ_rainfall_max_6h',
'HAZ_rainfall_max_24h','HAZ_v_max', 'HAZ_dis_track_min','GEN_landslide_per', 'GEN_stormsurge_per',
'GEN_Bu_p_inSSA','GEN_Bu_p_LS', 'GEN_Red_per_LSbldg', 'GEN_Or_per_LSblg','GEN_Yel_per_LSSAb',
'GEN_RED_per_SSAbldg', 'GEN_OR_per_SSAbldg','GEN_Yellow_per_LSbl', 'TOP_mean_slope', 'TOP_mean_elevation_m',
'TOP_ruggedness_stdev', 'TOP_mean_ruggedness', 'TOP_slope_stdev','VUL_poverty_perc', 'GEN_with_coast',
'GEN_coast_length','VUL_Housing_Units', 'VUL_StrongRoof_StrongWall','VUL_StrongRoof_LightWall',
'VUL_StrongRoof_SalvageWall','VUL_LightRoof_StrongWall', 'VUL_LightRoof_LightWall','VUL_LightRoof_SalvageWall',
'VUL_SalvagedRoof_StrongWall','VUL_SalvagedRoof_LightWall', 'VUL_SalvagedRoof_SalvageWall','VUL_vulnerable_groups',
'VUL_pantawid_pamilya_beneficiary','DAM_perc_dmg']



df_total = df_total.filter(selected_columns)
    

In [ ]:
df_predisasters

### save final dataframe for model input

In [158]:
df_predisasters.to_csv("./data/model_input/df_predisasters.csv", index=False)

In [ ]:
np.mean(df_total["DAM_perc_dmg"])

In [97]:
df_total=df_total[df_total["DAM_perc_dmg"].notnull()]

In [ ]:
len(df_total)

In [ ]:
np.mean(df_total["DAM_perc_dmg"])